# Categorical Transformation for DL
> a list of things to categoryical transformation

## Indexing forward and backward

In [6]:
from forgebox.category import Category
from forgebox.imports import *

In [2]:
cates = Category(list(map(lambda x:f"Cate_{x+1}",range(50))))

In [3]:
cates

Category Manager with 50

In [4]:
cates.i2c[:5]

array(['Cate_1', 'Cate_2', 'Cate_3', 'Cate_4', 'Cate_5'], dtype='<U7')

In [7]:
test_c = np.random.randint(1,50,1000)

### Indices to categories

In [8]:
labels = cates.i2c[test_c]
labels[:20]

array(['Cate_46', 'Cate_10', 'Cate_25', 'Cate_43', 'Cate_20', 'Cate_30',
       'Cate_39', 'Cate_15', 'Cate_31', 'Cate_3', 'Cate_44', 'Cate_2',
       'Cate_29', 'Cate_24', 'Cate_48', 'Cate_34', 'Cate_48', 'Cate_11',
       'Cate_5', 'Cate_32'], dtype='<U7')

### Category to indices

In [9]:
cates.c2i[labels[:20]]

array([45,  9, 24, 42, 19, 29, 38, 14, 30,  2, 43,  1, 28, 23, 47, 33, 47,
       10,  4, 31])

## With missing tokens

We can set pad_mst to True to manage missing token

In [10]:
nt = Category("ATCG", pad_mst=True)

### Categories to indices

In [11]:
nt.c2i[list("AAACCTTATTGCAGCOAAT")]

array([1, 1, 1, 3, 3, 2, 2, 1, 2, 2, 4, 3, 1, 4, 3, 0, 1, 1, 2])

### Indices to categories

In [12]:
nt.i2c[[1, 1, 1, 3, 3, 2, 2, 2, 2, 4, 3, 1, 4, 3, 0, 1, 1, 2]]

array(['A', 'A', 'A', 'C', 'C', 'T', 'T', 'T', 'T', 'G', 'C', 'A', 'G',
       'C', '[MST]', 'A', 'A', 'T'], dtype='<U5')

## Data save and load

### Save categories

In [13]:
nt.save("atcg.json")

### Load categories

In [14]:
cm = Category.load("atcg.json")
cm

Category Manager with 5